Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *
from tickers import tickers

from scipy.stats import norm as norm
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import load_workbook
from yahoo_query import *

In [2]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

# stock_list = pd.read_csv('optionablestocks.csv')['OPTION SYMBOL'].tolist()
# stock_list = list(set(tickers[3:] + stock_list))

cad_names = pd.read_csv('cad_names.csv')['Symbol'].tolist()
us_names = pd.read_csv('us_names.csv')['Symbol'].tolist()
start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info_quarter = yahoo_data.earnings_quarterly.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
    
    annual_info = yahoo_data.cashFlowStatementAnnual.join(yahoo_data.incomeStatementAnnual.drop(['netIncome','maxAge'],
                                                                                                  axis = 1),
                                                           rsuffix='_income').join(yahoo_data.balanceSheetAnnual,
                                                                                   rsuffix = '_balance')
    annual_info['earnings'] = yahoo_data.earnings_annual.sort_index(ascending = False)['earnings'].tolist()
#     earnings_info['earnBeatsBefore'] = 0
#     earnings_info['earnMissBefore'] = 0

#     for idx, row in earnings_info.iterrows():
#         earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
#         earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
#     earnings_info = earnings_info.shift(1)


#     earnings_moves = past_earnings(ticker).sort_index()
#     earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
#                                     (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


#     earnings_df = pd.concat([earnings_info.reset_index(), 
#                              earnings_moves.reset_index()], axis = 1)
    earnings_info_quarter.columns = ['quarter' if col == 'index' else col for col in earnings_info_quarter.columns.tolist()]
    earnings_info_quarter['Underlying'] = ticker
    annual_info.columns = ['year' if col == 'index' else col for col in annual_info.columns.tolist()]
    annual_info['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_info_quarter, annual_info, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    annual_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_annual_info, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            annual_lst.append(curr_annual_info)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_annual_info, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    annual_lst.append(curr_annual_info)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    annual_df = pd.concat(annual_lst, axis = 0)
    #earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, annual_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [ ]:
earnings_df, annual_df, keyStats_df, failed_list = download_yahoo_data(us_names, retries = 3)

datenow = dt.datetime.today().strftime("%Y-%m-%d")
earnings_df.to_csv('us_quarterly_{}.csv'.format(datenow))
annual_df.to_csv('us_annual_{}.csv'.format(datenow))
keyStats_df.to_csv('us_keystats_{}.csv'.format(datenow))


In [ ]:
earnings_df, annual_df, keyStats_df, failed_list = download_yahoo_data(cad_names, retries = 3)

datenow = dt.datetime.today().strftime("%Y-%m-%d")
earnings_df.to_csv('cad_quarterly_{}.csv'.format(datenow))
annual_df.to_csv('cad_annual_{}.csv'.format(datenow))
keyStats_df.to_csv('cad_keystats_{}.csv'.format(datenow))

In [ ]:
# earnings, ks, failed_list = download_yahoo_data(stock_list, retries = 10)
# earnings = earnings.drop(earnings.isnull().sum().sort_values().tail(13).index.tolist(), axis = 1)
# earnings.to_csv('full_data.csv')

fs_df = pd.read_csv('full_data.csv', index_col = 0)

fs = fs_df[['Underlying']]
fs['curr_ratio'] = fs_df['totalCurrentAssets']/fs_df['totalCurrentLiabilities']
fs['totaldebtequity'] = fs_df['totalLiab']/fs_df['totalStockholderEquity']
fs['longdebtequity'] = fs_df['longTermDebt']/fs_df['totalStockholderEquity']
fs['shortdebtequity'] = fs_df['shortLongTermDebt']/fs_df['totalStockholderEquity']
fs['daySalesOut'] = (fs_df['netReceivables']/fs_df['totalRevenue'].replace(0,np.nan))*365
fs['intangToBook'] = fs_df['intangibleAssets']/fs_df['totalStockholderEquity']
fs['invToRev'] = fs_df['inventory']/fs_df['totalRevenue'].replace(0,np.nan)
fs['ltdebtToCap'] = fs_df['longTermDebt']/fs_df['investments']
fs['stdebtToCap'] = fs_df['shortLongTermDebt']/fs_df['investments']
fs['ltdebtPercent'] = fs_df['longTermDebt']/fs_df['totalLiab']
fs['stdebtPercent'] = fs_df['shortLongTermDebt']/fs_df['totalLiab']

fs['grossMargin'] = fs_df['grossProfit']/fs_df['totalRevenue'].replace(0,np.nan)
fs['profitMargin'] = fs_df['netIncome']/fs_df['totalRevenue'].replace(0,np.nan)
fs['operatingMargin'] = fs_df['operatingIncome']/fs_df['totalRevenue'].replace(0,np.nan)

fs['operatingCashflow'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalRevenue'].replace(0,np.nan)
fs['assetEfficiency'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalAssets']
fs['currLiabCoverage'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalCurrentLiabilities']
fs['ltDebtCoverage'] = fs_df['totalCashFromOperatingActivities']/fs_df['longTermDebt']
fs['cashGeneratingPower'] = fs_df['totalCashFromOperatingActivities']/(fs_df['totalCashFromOperatingActivities'] + fs_df['totalCashFromFinancingActivities'] + fs_df['totalCashflowsFromInvestingActivities'])

fs_pct = []
for name in fs.Underlying.drop_duplicates().tolist():
    pct = fs[fs['Underlying'] == name].sort_index()
    pct[pct.columns.tolist()[1:]] = pct[pct.columns.tolist()[1:]].pct_change()
    fs_pct.append(pct.tail(len(pct)-1))

fs_pct = pd.concat(fs_pct, axis = 0)

In [ ]:
filename = 'Leaps Filtering.xlsx'
ks = pd.read_excel(filename, sheetname = 'keystats', index_col = 0).drop_duplicates()

book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

fs_pct.drop_duplicates().to_excel(writer, "finpctdata")
fs.drop_duplicates().to_excel(writer,"findata")
ks.drop('industry',axis = 1).groupby('sector').mean().to_excel(writer, "keystatsAvg")
writer.save()

## Checking Key Stats on Single Names
### For each industry filter names that have:
1. positive earnings growth
2. positive profit margin
3. lower than industry average p/b, p/e, ev/ebitda, debt/equity
4. rising margins quarter over quarter
5. decreasing debt/equity quarter over quarter
6. decreasing days of sales outstanding
7. lower than industry average inventory/revenue ratio

In [ ]:
filename = 'Leaps Filtering.xlsx'
ks = pd.read_excel(filename, sheetname = 'keystats', index_col = 0).drop_duplicates()
ksavg = pd.read_excel(filename, sheetname = 'keystatsAvg', index_col = 0).drop_duplicates()
finpct = pd.read_excel(filename, sheetname = 'finpctdata', index_col = 0).drop_duplicates().fillna(0)
fin = pd.read_excel(filename, sheetname = 'findata', index_col = 0).drop_duplicates()

# Creating scores of pct changes
tickers = finpct.Underlying.drop_duplicates().tolist()
columns = finpct.columns.tolist()[1:]
values = ['inc','dec','dec','dec','dec','flat','dec','dec','dec',
          'dec','dec','inc','inc','inc','inc','inc','inc','inc',
          'inc']
field_directions = dict(zip(columns,values))
scores = pd.DataFrame(index = tickers, columns = columns)

for name in tickers:
    for field in columns:
        curr_fin = finpct[(finpct['Underlying'] == name)][field]
        if len(curr_fin) > 1:
            if field_directions[field] == 'inc':
                point = len(curr_fin) == sum(curr_fin >= 0)
            elif field_directions[field] == 'dec':
                point = len(curr_fin) == sum(curr_fin <= 0)
            else:
                point = False
        else:
            point = False
            
        scores.loc[name,field] = point*1
        
scores['Score'] = scores.sum(axis = 1)

In [ ]:
top_names = scores[scores['Score'] >= 10].sort_values('Score', ascending = False)
ks_col_order = ['currentPrice', 'sector', 'industry',
                '52WeekChange', 'SandP52WeekChange', 'beta', 'bookValue',
                'currentRatio', 'debtToEquity', 'earningsGrowth', 'revenueGrowth', 
                'returnOnAssets', 'returnOnEquity',
                'earningsQuarterlyGrowth', 'heldPercentInsiders', 'heldPercentInstitutions', 
                'sharesPercentSharesOut', 'shortPercentOfFloat', 
                'ebitda', 'ebitdaMargins',
                'enterpriseToEbitda', 'enterpriseToRevenue', 'enterpriseValue',
                'floatShares', 'forwardEps', 'forwardPE', 'freeCashflow',
                'fullTimeEmployees', 'grossMargins', 'grossProfits',
                'netIncomeToCommon', 'operatingCashflow', 'profitMargins','operatingMargins',
                'overallRisk', 'pegRatio', 'priceToBook', 
                'quickRatio', 'recommendationKey',
                'recommendationMean', 
                'revenuePerShare', 'sharesOutstanding',
                'sharesShort', 'sharesShortPriorMonth',
                'shortRatio', 'targetHighPrice',
                'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'totalCash',
                'totalCashPerShare', 'totalDebt', 'totalRevenue', 'trailingEps']

In [ ]:
filteredfilename = 'filtered leaps.xlsx'
top_names.to_excel(filteredfilename,'scores')

book = load_workbook(filteredfilename)
writer = pd.ExcelWriter(filteredfilename, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

finpct[finpct.Underlying.isin(top_names.index.tolist())].to_excel(writer, 'finpctdata')
fin[fin.Underlying.isin(top_names.index.tolist())].to_excel(writer, 'findata')
ks[ks_col_order][ks.index.isin(top_names.index.tolist())].to_excel(writer,'keystats')
ks.drop('industry',axis = 1).groupby('sector').mean().to_excel(writer, "keystatsAvg")

writer.save()

